In [1]:
%autoreload 2

In [2]:
from argparse import Namespace
from collections import defaultdict
import copy
from datetime import datetime
import difflib
import duckdb
from functools import reduce
import glob
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML  # type: ignore
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tatsu, tatsu.ast
import tqdm.notebook as tqdmn
import sqlparse


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../reward-machine'))

import compile_predicate_statistics
import compile_predicate_statistics_split_args
from compile_predicate_statistics_split_args import *
from config import SPECIFIC_NAMED_OBJECTS_BY_ROOM
import config
from utils import get_object_assignments, extract_variables

import ast_printer
import ast_parser
import ast_utils

2024-05-07 17:38:05 - ast_utils - DEBUG    - Using cache folder: /Users/guydavidson/tmp/game_generation_cache


In [3]:
cache_dir = compile_predicate_statistics.get_project_dir() + '/reward-machine/caches'

bitstings_df_path = os.path.join(cache_dir, 'predicate_statistics_bitstring_intervals_028b3733.pkl.gz')
bitstrings_df = pd.read_pickle(bitstings_df_path)
print(bitstrings_df.shape)
with gzip.open(os.path.join(cache_dir, 'trace_lengths_028b3733.pkl'), 'rb') as f:
    trace_lengths_and_domains = pickle.load(f)


# regular_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics.pkl'))
split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_028b3733.pkl.gz'))
# split_args_df = split_args_df[split_args_df['predicate'] != 'same_type']
# print(split_args_df.shape)
# # split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_4d5dd602.pkl.gz'))

# # stats = compile_predicate_statistics.CommonSensePredicateStatistics(cache_dir)
# split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(
#     # cache_dir, compile_predicate_statistics_split_args.CURRENT_TEST_TRACE_NAMES, overwrite=False
#     )

(2231745, 13)


In [ ]:
# start = 855
# end = 856
# def find_index_ranges(string):
#     ranges = []
#     start = None
#     for i, char in enumerate(string):
#         if char == '1':
#             if start is None:
#                 start = i
#         else:
#             if start is not None:
#                 ranges.append((start, i))
#                 start = None
#     if start is not None:
#         ranges.append((start, len(string)))
#     return ranges

# relevant_rows = (bitstrings_df.trace_id == 'G2chsLHjitbS4fASq4vq-preCreateGame-rerecorded') & (bitstrings_df.predicate == 'on') & (bitstrings_df.arg_1_id == 'Drawer|-01.52|+00.14|+00.35') & (bitstrings_df.arg_2_id == 'Golfball|+00.96|+01.04|-02.70')
# d = bitstrings_df[relevant_rows]
# print(d.shape)
# intervals = d.iloc[0].intervals
# print(find_index_ranges(intervals))
# intervals = intervals[:start] + '1' * (end - start) + intervals[end:]
# print(find_index_ranges(intervals))
# bitstrings_df.loc[relevant_rows, 'intervals'] = intervals

# bitstrings_df.to_pickle(bitstings_df_path)

In [ ]:
# changed = False

# for i in (1, 2):
#     rows = (bitstrings_df[f'arg_{i}_id'] == 'SmallSlide|-01.31|+00.14|-03.10') & (bitstrings_df[f'arg_{i}_type'] == 'triangle_block_tan')
#     unique_types = bitstrings_df.loc[rows][f'arg_{i}_type'].unique()
#     unique_is_blocks = bitstrings_df.loc[rows][f'arg_{i}_is_block'].unique()
#     print(bitstrings_df.loc[rows, [f'arg_{i}_type', f'arg_{i}_is_block']].shape, unique_types, unique_is_blocks)
#     if len(unique_types) == 1 and unique_types[0] == 'triangle_block_tan':
#         print('Editing type to triangular_ramp_tan')
#         bitstrings_df.loc[rows, f'arg_{i}_type'] = 'triangular_ramp_tan'

#     if len(unique_is_blocks) == 1 and unique_is_blocks[0] == True:
#         print('Editing is_block to False')
#         bitstrings_df.loc[rows, f'arg_{i}_is_block'] = False
#         changed = True

#         unique_types = bitstrings_df.loc[rows][f'arg_{i}_type'].unique()
#         unique_is_blocks = bitstrings_df.loc[rows][f'arg_{i}_is_block'].unique()
#         print(bitstrings_df.loc[rows, [f'arg_{i}_type', f'arg_{i}_is_block']].shape, unique_types, unique_is_blocks)
#         print()

# for i in (1, 2):
#     rows = (bitstrings_df[f'arg_{i}_id'] == 'TriangleBlock|-02.94|+01.23|-02.46')
#     unique_types = bitstrings_df.loc[rows][f'arg_{i}_type'].unique()
#     print(bitstrings_df.loc[rows].shape, unique_types)

#     if len(unique_types) == 1:
#         partial_copy = bitstrings_df.loc[rows].copy()
#         partial_copy[f'arg_{i}_type'] = 'triangle_block_tan'
#         bitstrings_df = pd.concat([bitstrings_df, partial_copy])  # type: ignore
#         changed = True

#         rows = bitstrings_df[f'arg_{i}_id'] == 'TriangleBlock|-02.94|+01.23|-02.46'
#         type_value_counts = bitstrings_df.loc[rows][f'arg_{i}_type'].value_counts()
#         print(bitstrings_df.loc[rows].shape, type_value_counts)
#         print()

# if changed:
#     bitstrings_df.to_pickle(bitstings_df_path)

In [ ]:
# drop_index = bitstrings_df[bitstrings_df.arg_1_type.isin(config.ON_EXCLUDED_OBJECT_TYPES) & (bitstrings_df.predicate == 'on')].index
# filtered_bitstrings_df = bitstrings_df.drop(drop_index)
# filtered_bitstrings_df = filtered_bitstrings_df.reset_index(drop=True)

# print(filtered_bitstrings_df.shape, bitstrings_df.shape, 
#       bitstrings_df.shape[0] - filtered_bitstrings_df.shape[0], 
#       filtered_bitstrings_df.index.max())

# filtered_bitstrings_df.to_pickle(bitstings_df_path)

In [ ]:
# sorted_bitstrings_df = bitstrings_df.sort_values(by=['predicate', 'arg_1_type', 'arg_2_type', 'trace_id'])
# sorted_bitstrings_df.to_pickle(bitstings_df_path)

In [ ]:
# game_object_excluded_types = set(config.GAME_OBJECT_EXCLUDED_TYPES)
# bitstrings_df = bitstrings_df.assign(arg_1_is_game_object=~bitstrings_df.arg_1_type.isin(game_object_excluded_types), arg_2_is_game_object=~bitstrings_df.arg_2_type.isin(game_object_excluded_types))
# bitstrings_df.to_pickle(bitstings_df_path)

In [ ]:
# bitstrings_df = bitstrings_df.assign(arg_1_is_block=bitstrings_df.arg_1_type.str.contains('block'), arg_2_is_block=bitstrings_df.arg_2_type.str.contains('block'))
# bitstrings_df.to_pickle(bitstings_df_path)


In [ ]:
b = bytes([0, 1])
BYTE_MAPPING = {b: str(i) for i, b in enumerate(b)}


def row_to_string_intervals(row):
    value = np.zeros(row['trace_length'], dtype=np.uint8)
    for interval in row['intervals']:
        value[interval[0]:interval[1]] = 1

    return ''.join(map(lambda b: BYTE_MAPPING[b], value.tobytes()))
    

def create_bitstings_df(df, trace_lengths_and_domains_dict, output_path):
    trace_lengths_and_domains_rows = [(key, *value) for key, value in trace_lengths_and_domains_dict.items()]
    trace_lengths_and_domains_df = pd.DataFrame(trace_lengths_and_domains_rows, columns=['trace_id', 'trace_length', 'domain'])

    split_args_with_trace_length_df = df.join(trace_lengths_and_domains_df.drop(columns=['domain']).set_index('trace_id'), on='trace_id')
    split_args_with_string_intervals_df = split_args_with_trace_length_df.assign(intervals=split_args_with_trace_length_df.apply(row_to_string_intervals, axis=1))
    split_args_with_string_intervals_df.drop(columns=['trace_length']).to_pickle(output_path)
    

In [ ]:
trace_lengths_and_domains_rows = [(key, *value) for key, value in trace_lengths_and_domains.items()]
trace_lengths_and_domains_df = pd.DataFrame(trace_lengths_and_domains_rows, columns=['trace_id', 'trace_length', 'domain'])

split_args_with_trace_length_df = split_args_df.join(trace_lengths_and_domains_df.drop(columns=['domain']).set_index('trace_id'), on='trace_id')

In [ ]:
b = bytes([0, 1])
BYTE_MAPPING = {b: str(i) for i, b in enumerate(b)}


def row_to_string_intervals(row):
    value = np.zeros(row['trace_length'], dtype=np.uint8)
    for interval in row['intervals']:
        value[interval[0]:interval[1]] = 1

    return ''.join(map(lambda b: BYTE_MAPPING[b], value.tobytes()))
    # return np.array2string(value, separator='', threshold=max_length + 10)[1:-1].replace('\n ', '')




In [ ]:
split_args_with_string_intervals_df = split_args_with_trace_length_df.assign(intervals=split_args_with_trace_length_df.apply(row_to_string_intervals, axis=1))
split_args_with_string_intervals_df

In [ ]:
split_args_with_string_intervals_df.to_pickle(bitstings_df_path)

In [ ]:
split_args_with_string_intervals_df.loc[
    (split_args_with_string_intervals_df.arg_1_type == 'ball'), "arg_1_type"]

In [ ]:
for arg_id, arg_type in compile_predicate_statistics_split_args.OBJECT_ID_TYPE_REMAP.items():
    original_type = None
    if 'window' in arg_id.lower():
        original_type = 'window'
    if 'shelf' in arg_id.lower():
        original_type = 'shelf'
    
    if original_type is None:
        raise ValueError(f'Could not find original type for {arg_id}')
    
    split_args_with_string_intervals_df.loc[
    (split_args_with_string_intervals_df.arg_1_id == arg_id) & (split_args_with_string_intervals_df.arg_1_type == original_type), "arg_1_type"] = arg_type

    split_args_with_string_intervals_df.loc[
    (split_args_with_string_intervals_df.arg_2_id == arg_id) & (split_args_with_string_intervals_df.arg_2_type == original_type), "arg_2_type"] = arg_type

In [ ]:
from itertools import chain
from config import OBJECTS_BY_ROOM_AND_TYPE, SPECIFIC_NAMED_OBJECTS_BY_ROOM, META_TYPES, GAME_OBJECT, BUILDING

all_df_types = set(split_args_with_string_intervals_df.arg_1_type.unique()) | set(split_args_with_string_intervals_df.arg_2_type.unique())
computed_types = set(reduce(lambda x, y: x + y, [list(x.keys()) for x in chain(OBJECTS_BY_ROOM_AND_TYPE.values(), SPECIFIC_NAMED_OBJECTS_BY_ROOM.values())]))
computed_types.difference_update(META_TYPES.keys())
computed_types.remove(GAME_OBJECT)
computed_types.add(BUILDING)

In [ ]:
all_df_types - computed_types

In [ ]:
computed_types - all_df_types

In [ ]:
RENAMED_TYPES = """blue_cube_block
tan_cube_block
yellow_cube_block
blue_pyramid_block
red_pyramid_block
yellow_pyramid_block
blue_dodgeball
red_dodgeball
pink_dodgeball
green_golfball
green_triangular_ramp""".split('\n')

class DefaultValueDict(dict):
    def __init__(self, *args, **kawags):
        super().__init__(*args, **kawags)

    def __missing__(self, key):
        return key
    
arg_type_mapping = DefaultValueDict()
for renamed_type in RENAMED_TYPES:
    sp = renamed_type.split('_')
    new_name = '_'.join(sp[1:] + sp[:1])
    arg_type_mapping[renamed_type] =  new_name



bitstrings_df.assign(arg_1_type=bitstrings_df.arg_1_type.map(arg_type_mapping), 
                     arg_2_type=bitstrings_df.arg_2_type.map(arg_type_mapping),).to_pickle(bitstings_df_path)

In [ ]:
MAX_LENGTH = split_args_stats.trace_lengths_and_domains_df.select(pl.col('trace_length').max()).item()
print(MAX_LENGTH)
b = bytes([0, 1])
BYTE_MAPPING = {b: str(i) for i, b in enumerate(b)}



def intervals_to_string(intervals, max_length: int = MAX_LENGTH):
    value = np.zeros(max_length, dtype=np.uint8)
    for interval in intervals:
        value[interval[0]:interval[1]] = 1

    return ''.join(map(lambda b: BYTE_MAPPING[b], value.tobytes()))
    # return np.array2string(value, separator='', threshold=max_length + 10)[1:-1].replace('\n ', '')


intervals = split_args_df.intervals.apply(intervals_to_string)

# small_split_args_df = small_split_args_df.assign()

In [ ]:
from src.ast_utils import cached_load_and_parse_games_from_file
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='.'))

In [ ]:
from fitness_features import *

class SmallLogicalsRefactorTestFeaturizer(PredicateFoundInDataSmallLogicals):
    def __init__(self, use_trace_ids: bool = True):
        super().__init__()
        self.use_trace_ids = use_trace_ids
        self.current_implementation_more = defaultdict(dict)
        self.refactored_implementation_more = defaultdict(dict)

    def _query_predicate_data_estimator(self, pred: tatsu.ast.AST, 
                                        mapping: typing.Dict[str, typing.Union[str, typing.List[str]]],
                                        context: ContextDict) -> typing.Optional[int]:

        current_results = self.predicate_data_estimator.filter(pred, mapping, return_trace_ids=self.use_trace_ids)

        # refactored_method = self.predicate_data_estimator._handle_and_refactored if 'and_args' in pred else self.predicate_data_estimator._handle_or_refactored
        # refactored_query = refactored_method(pred, mapping, return_trace_ids=self.use_trace_ids)[0]
        # refactored_results = self.predicate_data_estimator.con.execute(refactored_query).fetchall()
        refactored_results = self.predicate_data_estimator.filter(pred, mapping, return_trace_ids=self.use_trace_ids, use_refactored_impl=True)

        if self.use_trace_ids:
            current_results = set(current_results)
            refactored_results = set(itertools.chain.from_iterable(refactored_results))

            if current_results.symmetric_difference(refactored_results):
                pred_str = ast_printer.ast_section_to_string(pred, context[SECTION_CONTEXT_KEY]) + '_' + str(mapping) # type: ignore
                only_current_trace_ids = current_results - refactored_results
                only_refactored_trace_ids = refactored_results - current_results

                rule = pred.parseinfo.rule  # type: ignore

                if only_current_trace_ids and pred_str not in self.current_implementation_more:
                    self.current_implementation_more[rule][pred_str] = only_current_trace_ids

                if only_refactored_trace_ids and rule not in self.refactored_implementation_more:
                    self.refactored_implementation_more[rule][pred_str] = only_refactored_trace_ids
                    
        else:
            if current_results != refactored_results:
                pred_str = ast_printer.ast_section_to_string(pred, context[SECTION_CONTEXT_KEY]) + '_' + str(mapping)
                rule = pred.parseinfo.rule  # type: ignore
                result_set = self.current_implementation_more if current_results > refactored_results else self.refactored_implementation_more
                result_set[rule][pred_str] = current_results - refactored_results

args = Namespace(
    no_binarize=False, 
    no_merge=False, 
    use_specific_objects_ngram_model=False,
    include_predicate_under_modal_terms=False,
    include_arg_types_terms=False,
    include_compositionality_terms=False,
)
preprocessors = []

if not args.no_binarize:
    preprocessors.append(BinarizeFitnessFeatures())

if not args.no_merge and args.include_arg_types_terms:  # the merge is only used for the arg_types featuers
    preprocessors.append(MergeFitnessFeatures(COMMON_SENSE_PREDICATES_FUNCTIONS))


In [ ]:
fitness = ASTFitnessFeaturizer(args, preprocessors=preprocessors)
refactor_test = SmallLogicalsRefactorTestFeaturizer(False)
fitness.register(refactor_test)

In [ ]:
_ = [fitness.parse(game_asts[i], 'interactive-beta.pddl', return_row=False) for i in range(len(game_asts))]


print('Current implementation more:')
for rule, rule_dict in refactor_test.current_implementation_more.items():
    print(f'Under rule {rule}:')
    for pred_str, diff in rule_dict.items():
        if isinstance(diff, set):
            diff = len(diff)
        print(f'{pred_str} => {diff}')

print('Refactored implementation more:')
for rule, rule_dict in refactor_test.refactored_implementation_more.items():
    print(f'Under rule {rule}:')
    for pred_str, diff in rule_dict.items():
        if isinstance(diff, set):
            diff = len(diff)
        print(f'{pred_str} => {diff}')


In [4]:
import compile_predicate_statistics_full_database

stats = compile_predicate_statistics_full_database.CommonSensePredicateStatisticsFullDatabase.get_instance(force_trace_names_hash='028b3733', log_queries=True)

2024-05-07 17:38:36 - compile_predicate_statistics_full_database - INFO     - Loading data from files with query timeout 15
2024-05-07 17:38:58 - compile_predicate_statistics_full_database - INFO     - Creating DuckDB table...
2024-05-07 17:38:59 - compile_predicate_statistics_full_database - INFO     - Creating data table indices...
2024-05-07 17:39:21 - compile_predicate_statistics_full_database - INFO     - Loaded data, found 843825 rows


In [10]:
from src.ast_utils import cached_load_and_parse_games_from_file
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='.'))

test_game_str = """
(define (game test-game) (:domain many-objects-room-v1)
(:constraints (and
    (preference testPred
        (exists (?v1 - building ?v2 - block)
            (at-end (or  
                (and (adjacent ?g agent) (adjacent door agent) (agent_holds ?d))
                (near door ?g)
                (and (not (agent_holds ?v0)) (in_motion ?v0))
                (and (not (in_motion ?v3)) (in ?v2 ?v3) (on ?v2 ?v3)) 
                
                ; (agent_holds ?v1)
                ; (and (not (agent_holds ?v1)) (in_motion ?v1))
                ; (and (not (in_motion ?v1)) (in ?v0 ?v1))

                ; (adjacent bed agent)
                ; (on bed ?v2)
                ; (on bed agent)  
                ; (in_motion ?r)  
                ; (not    (agent_holds ?r) )  
                ; (not    (touch floor ?r) )
                ; (not    (in_motion ?d) )  
                ; (not    (in_motion ?b) )  
                ; (not    (object_orientation ?t upright) )
                ; (in_motion ?d)
                ; (not (agent_holds ?d))
                ; (not (agent_holds ?t))
                ; (adjacent agent bed)
                ; (not (agent_holds ?v1))
                ; (not (on bed ?v2))
            ))
        )
    )
))
(:scoring (+
    (* (count testPred) 1)
)))
""".strip()
# test_game_str = """
# (define (game test-game) (:domain many-objects-room-v1)
# (:constraints (and
#     (preference testPred
#         (exists (?v0 - block ?v1 - wall)
#             (at-end (and 
#                 (on ?v1 ?v0)
#                 (in ?v0 ?v1)
#             ))
#         )
#     )
# ))
# (:scoring (+
#     (* (count testPred) 1)
# )))
# """.strip()

test_ast = grammar_parser.parse(test_game_str)
test_pred = test_ast[3][1].preferences[0].definition.pref_body.body.exists_args.at_end_pred.pred
print(ast_printer.ast_section_to_string(test_pred, ast_parser.PREFERENCES))

2024-05-07 17:41:48 - src.ast_utils - INFO     - Loading from cache file: /Users/guydavidson/tmp/game_generation_cache/interactive-beta-cache.pkl.gz
2024-05-07 17:41:48 - src.ast_utils - INFO     - Finished loading cache file: /Users/guydavidson/tmp/game_generation_cache/interactive-beta-cache.pkl.gz


(or  (and    (adjacent ?g agent)    (adjacent door agent)    (agent_holds ?d) )  (near door ?g)  (and    (not      (agent_holds ?v0)   )    (in_motion ?v0) )  (and    (not      (in_motion ?v3)   )    (in ?v2 ?v3)    (on ?v2 ?v3) ))


In [9]:
print(ast_printer.ast_section_to_string(test_pred.or_args[0], ast_parser.PREFERENCES))

(and  (adjacent ?g agent)  (adjacent door agent)  (agent_holds ?d))


In [14]:
mapping = {'?g': ['golfball_green'], '?d':  ['dodgeball']}
stats.filter(test_pred.or_args[1], mapping, return_full_result=True)

,trace_id,domain,intervals,door,?g
0,5217Qbj8V5farCF3bBJT-createGame-rerecorded,many,b'\x07\xfe\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
1,FyGQn1qJCLTLU1hfQfZ2-freePlay-rerecorded,many,b'\x05\xf8\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
2,FyGQn1qJCLTLU1hfQfZ2-gameplay-attempt-1-rereco...,many,b'\x01\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
3,ShyRlIFgTWW6vqNrd3K5-gameplay-attempt-1-rereco...,many,b'\x06\xfc\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
4,Q6a8AbiIdcLA9tJzAu14-gameplay-attempt-1-rereco...,many,b'\x01\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
5,5217Qbj8V5farCF3bBJT-gameplay-attempt-1-rereco...,many,b'\x05\xf8\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
6,IvoZWi01FO2uiNpNHyci-gameplay-attempt-2-rereco...,many,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
7,SFpazSkgQ7MFSwDEa3c9-gameplay-attempt-1-rereco...,many,b'\x06\xfc\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
8,YB9HWpd4rMWwS3P2mCVk-freePlay-rerecorded,many,b'\x02\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70
9,gPJmGRgPdHSlBI7IMc3z-gameplay-attempt-2-rereco...,many,b'\x04\xf0\x00\x00\x00\x00\x00\x00\x00\x00\x00...,door,Golfball|+01.05|+01.04|-02.70


In [ ]:
from reward_machine_trace_filter import PreferenceStateMachineLogicEvaluator, _query_trace_id_to_length, MERGE_IGNORE_COLUMNS
logic_evaluator = PreferenceStateMachineLogicEvaluator(_query_trace_id_to_length(stats.con))


mapping = {'?v0': ['ball'], '?v1': ['cylindrical_block'], '?v2': ['cylindrical_block']}
# trace_id = 'G2chsLHjitbS4fASq4vq-preCreateGame-rerecorded'
dfs = []
for pred in test_pred.or_args:
    df = stats.filter(pred, mapping, return_full_result=True)
    # df = df[df.trace_id == trace_id]
    dfs.append(df)


dfs = [logic_evaluator._preprocess_results(df) for df in dfs]

In [ ]:
merged_df = None
wildcard_indices = []
merged_indices = []

initial_intervals_index = None
for i, result in enumerate(dfs):
    if isinstance(result, pd.DataFrame):
        if merged_df is None:
            merged_df = result
            initial_intervals_index = i
        else:
            merge_columns = ['trace_id']
            merge_columns.extend(set(merged_df.columns) & set(result.columns) - MERGE_IGNORE_COLUMNS)
            merged_df = merged_df.merge(result, on=merge_columns, how='inner', suffixes=('', f'_{i + 1}'))
            merged_indices.append(i)
    else:
        wildcard_indices.append(i)

merged_df.rename(columns={'intervals': f'intervals_{initial_intervals_index + 1}'}, inplace=True)

merged_df

In [ ]:
def _state_machine_logic_apply(
        row: pd.Series, modals: typing.List[str], trace_ids_found: typing.Set[str],
        check_all_unique_objects_per_trace: bool = False):

    if row.trace_id in trace_ids_found and not check_all_unique_objects_per_trace:
        return True

    index = 0
    state = 0
    interval_keys = sorted([key for key in row.keys() if key.startswith('intervals')])
    final_state = len(interval_keys)
    intervals = [row[key] for key in interval_keys]

    try:
        while index < logic_evaluator.trace_id_to_length[row.trace_id] and state < final_state:
            next_valid = intervals[state][index] == '1'
            if next_valid:
                print(f'At index {index} of trace {row.trace_id}, moved to state {state + 1}')
                state += 1

            elif state > 0 and (state == 1 or modals[state - 1] == 'hold'):
                current_valid = intervals[state - 1][index] == '1'
                if not current_valid:
                    print(f'At index {index} of trace {row.trace_id}, reset state to 0')
                    state = 0

            index += 1

    except IndexError:
        print(f'Trace id {row.trace_id} has length {logic_evaluator.trace_id_to_length[row.trace_id]}, but intervals {[len(i) for i in intervals]} for keys {interval_keys} are too short')
        raise

    result = state == final_state
    if result and not check_all_unique_objects_per_trace:
        trace_ids_found.add(row.trace_id)

    return result

for i, t in merged_df.iterrows():
    rv = _state_machine_logic_apply(t, ['once', 'hold', 'once'], set())
    if rv:
        break

In [ ]:
def find_index_ranges(string):
    ranges = []
    start = None
    for i, char in enumerate(string):
        if char == '1':
            if start is None:
                start = i
        else:
            if start is not None:
                ranges.append((start, i))
                start = None
    if start is not None:
        ranges.append((start, len(string)))
    return ranges


for i in range(3):
    print(find_index_ranges(merged_df.iloc[0][f'intervals_{i + 1}']))

In [ ]:
stats.con.execute(
"""
SELECT distinct(trace_id) FROM data
WHERE predicate='in'
AND arg_1_type='doggie_bed'
AND arg_2_type='dodgeball'

""").fetchdf()

In [ ]:
mapping = {'?v0': ['cylindrical_block_green']}
df1 = stats.filter(test_pred.or_args[0], mapping, return_full_result=True)
len(df1[df1.trace_id == 'GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rerecorded'].intervals.item())

In [ ]:
mapping = {'?v0': ['cylindrical_block_green']}
df2 = stats.filter(test_pred.or_args[1], mapping, return_full_result=True)
len(df2[df2.trace_id == 'GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rerecorded'].intervals.item())

In [ ]:
mapping = {'?v0': ['cylindrical_block_green']}
df3 = stats.filter(test_pred.or_args[2], mapping, return_full_result=True)
len(df3[df3.trace_id == 'GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rerecorded'].intervals.item())

In [ ]:
df2[df2.trace_id == 'GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rerecorded'].intervals.item()

In [ ]:
trace_id_query = """
SELECT trace_id, length from trace_length_and_domains tld
""".strip()

trace_id_to_length = {trace_id: length for trace_id, length in stats.con.execute(trace_id_query).fetchall()}

def intervals_to_strings_apply(row):
    length = trace_id_to_length[row.trace_id]
    return bin(int.from_bytes(row.intervals, 'big'))[-length:]

In [ ]:
mapping = {'?v2': ['cube_block']}

first_df = stats.filter(test_pred.or_args[0], mapping, return_full_result=True).drop(columns=['domain'])
first_df.intervals = first_df.apply(intervals_to_strings_apply, axis=1)

second_df = stats.filter(test_pred.or_args[1], mapping, return_full_result=True).drop(columns=['domain'])
second_df.intervals = second_df.apply(intervals_to_strings_apply, axis=1)

ignore_columns = set(['trace_id', 'domain', 'intervals'])

shared_variable_columns = set(first_df.columns) & set(second_df.columns) - ignore_columns
shared_variable_columns


In [ ]:
join_columns = ['trace_id']
join_columns.extend(shared_variable_columns)
merged_df = first_df.merge(second_df, on=join_columns, how='inner', suffixes=('_1', '_2'))

In [ ]:
def state_machine(row, found):
    if row.trace_id in found:
        return True
    
    index = 0
    state = 0
    interval_keys = sorted([key for key in row.keys() if key.startswith('intervals')])
    final_state = len(interval_keys)
    intervals = [row[key] for key in interval_keys]

    while index < trace_id_to_length[row.trace_id] and state != final_state:
        next_valid = intervals[state][index] == '1'
        if next_valid:
            state += 1

        elif state > 0:
            # TODO: add a check here that the current modal is a hold or that the current state is 1
            current_valid = intervals[state - 1][index] == '1'
            if not current_valid:
                state = 0

        index += 1

    result = state == final_state
    if result:
        found.add(row.trace_id)
    
    return result

In [ ]:
found = set()
merged_df[merged_df.apply(state_machine, axis=1, found=found)].trace_id.unique().tolist()

In [ ]:
def full_state_machine_logic(predicate_results: typing.List[typing.Union[s]]):
    pass

In [ ]:
b = b'23897234802374092187492364329784632789'

b2 = b'238972348023740921874923643297846327'

int(bin(int.from_bytes(b, 'big'))[-40:], 2)

In [ ]:
trace_id_query = """
SELECT trace_id, length from trace_length_and_domains tld
""".strip()

trace_id_to_length = {trace_id: length for trace_id, length in stats.con.execute(trace_id_query).fetchall()}

In [ ]:
trace_id_to_length['YB9HWpd4rMWwS3P2mCVk-freePlay-rerecorded']

In [ ]:
test_ast = grammar_parser.parse(test_game_str)
test_pred = test_ast[3][1].preferences[0].definition.pref_body.body.exists_args.at_end_pred.pred

mapping = {'?r': ['dodgeball_red']}

current_more_traces_permutations = []
refactored_more_traces_permutations = []
equal_permutations = []
current_count_by_permutation = {}
refactored_count_by_permutation = {}

children_key = 'and_args' if 'and_args' in test_pred else 'or_args'
original_children =  copy.deepcopy(test_pred[children_key])

for n_children in range(1, len(original_children) + 1):
    for combination in itertools.combinations(list(range(len(original_children))), n_children):
        new_children = []
        for i in combination:
            new_children.append(original_children[i])

        ast_utils.replace_child(test_pred, children_key, new_children)    

        current_results = set(stats.filter(test_pred, mapping, return_trace_ids=True))

        refactored_method = stats._handle_and_refactored if 'and_args' in test_pred else stats._handle_or_refactored
        refactored_query = refactored_method(test_pred, mapping, return_trace_ids=True)[0]

        try:
            refactored_results = stats.con.execute(refactored_query).fetchall()
            refactored_results = set(itertools.chain.from_iterable(refactored_results))
        
        except:
            print(sqlparse.format(refactored_query, reindent=True, keyword_case='upper'))
            raise

        current_count_by_permutation[combination] = len(current_results)
        refactored_count_by_permutation[combination] = len(refactored_results)

        sym_diff = refactored_results.symmetric_difference(current_results)
        if len(sym_diff) > 0:
            if len(current_results - refactored_results) > 0:
                current_more_traces_permutations.append(combination)
            if len(refactored_results - current_results) > 0:
                refactored_more_traces_permutations.append(combination)

        else:
            equal_permutations.append(combination)

if equal_permutations:
    print('equal combinations')
    for permutation in equal_permutations:
        print(permutation, current_count_by_permutation[permutation], refactored_count_by_permutation[permutation])

    print()

if current_more_traces_permutations:
    print('current more combinations')
    for permutation in current_more_traces_permutations:
        print(permutation, current_count_by_permutation[permutation], refactored_count_by_permutation[permutation])

    print()

if refactored_more_traces_permutations:
    print('refactored more combinations')
    for permutation in refactored_more_traces_permutations:
        print(permutation, current_count_by_permutation[permutation], refactored_count_by_permutation[permutation])

In [ ]:
return_trace_ids = False

mapping = {'?r': ['dodgeball_red']}
current_results = stats.filter(test_pred, mapping, return_trace_ids=return_trace_ids)
# df = stats.filter(test_pred, {'?v1': ['pillow'], '?v2': ['hexagonal_bin']}, last_interval_bit_set=True)
# df = stats.filter(test_pred, {'?v1': ['building'], '?v2': ['flat_block'], '?v3': ['block']}, last_interval_bit_set=True)
# df
if isinstance(current_results, int):
    print(current_results)
else:
    current_results = set(current_results)
    print(len(current_results))

In [ ]:
# mapping = {'?d': ['dodgeball'], '?b': ['beachball'], '?t': ['teddy_bear']}
refactored_method = stats._handle_and_refactored if 'and_args' in test_pred else stats._handle_or_refactored
refactored_query = refactored_method(test_pred, mapping, return_trace_ids=return_trace_ids)[0]

if not return_trace_ids:
    refactored_query = f"SELECT COUNT(*) FROM ({refactored_query} LIMIT 1);"

print(sqlparse.format(refactored_query, reindent=True, keyword_case='upper'))

refactored_results = stats.con.execute(refactored_query).fetchall()

if return_trace_ids:
    refactored_results = set(itertools.chain.from_iterable(refactored_results))
    print(len(refactored_results))
    print(refactored_results.symmetric_difference(current_results))

else:
    refactored_results = refactored_results[0][0]
    print(refactored_results, current_results)

In [ ]:
stats.con.execute(
"""
WITH predicate AS
  (SELECT d0.trace_id,
          d0.domain,
          d0.intervals,
          d0.arg_1_id AS "?r"
   FROM DATA d0
   WHERE d0.predicate='in_motion'
     AND (d0.arg_1_type='dodgeball_red')),
     negated_predicate AS
  (SELECT predicates.*,
          (COALESCE(intervals_d1, tld.intervals) | COALESCE(intervals_d2, tld.intervals)) AS intervals
   FROM
     (SELECT COALESCE(d1.trace_id, d2.trace_id) AS "trace_id",
             COALESCE(d1.domain, d2.domain) AS "domain",
             COALESCE(d1.arg_1_id, d2.arg_2_id) AS "?r",
             d2.arg_1_id AS "floor",
             d1.intervals AS intervals_d1,
             d2.intervals AS intervals_d2
      FROM
        (SELECT d1.trace_id,
                d1.domain,
                d1.intervals,
                d1.arg_1_id
         FROM DATA d1
         WHERE d1.predicate='agent_holds'
           AND (d1.arg_1_type='dodgeball_red')) d1
      FULL JOIN
        (SELECT d2.trace_id,
                d2.domain,
                d2.intervals,
                d2.arg_1_id,
                d2.arg_2_id
         FROM DATA d2
         WHERE d2.predicate='touch'
           AND (d2.arg_1_type='floor')
           AND (d2.arg_2_type='dodgeball_red')) d2 ON (d1.trace_id = d2.trace_id
                                                       AND (d1.arg_1_id = d2.arg_2_id))) predicates
   JOIN trace_length_and_domains tld ON predicates.trace_id = tld.trace_id),
     missing_negated_predicate AS
  (SELECT predicate.trace_id AS trace_id,
          o0.object_id AS "?r",
          o1.object_id AS "floor"
   FROM predicate
   JOIN object_type_to_id o0 ON predicate.domain = o0.domain
   AND o0.type = 'dodgeball_red'
   AND o0.object_id = predicate."?r"
   JOIN object_type_to_id o1 ON predicate.domain = o1.domain
   AND o1.type = 'floor'
   WHERE NOT EXISTS
       (SELECT 1
        FROM negated_predicate
        WHERE predicate.trace_id = negated_predicate.trace_id
          AND (o0.object_id = negated_predicate."?r"
               OR negated_predicate."?r" IS NULL)
          AND o1.object_id = negated_predicate."floor")
     AND NOT EXISTS
       (SELECT 1
        FROM negated_predicate
        WHERE predicate.trace_id = negated_predicate.trace_id
          AND (o0.object_id = negated_predicate."?r"
               OR o1.object_id = negated_predicate."floor")
          AND bit_count(~negated_predicate.intervals) = 0))

   
SELECT DISTINCT(trace_id) 
FROM predicate
WHERE bit_count(predicate.intervals) > 0 AND predicate.trace_id = 'SFpazSkgQ7MFSwDEa3c9-gameplay-attempt-1-rerecorded'
  AND (EXISTS
         (SELECT *
          FROM negated_predicate
          WHERE predicate.trace_id = negated_predicate.trace_id
            AND (predicate."?r" = negated_predicate."?r"
                 OR negated_predicate."?r" IS NULL)
            AND negated_predicate."floor" IS NOT NULL
            AND bit_count(predicate.intervals) > bit_count(predicate.intervals & negated_predicate.intervals) )
       OR EXISTS
         (SELECT *
          FROM missing_negated_predicate
          WHERE predicate.trace_id = missing_negated_predicate.trace_id )
)
LIMIT 1
""").fetchdf()

In [ ]:
stats.con.execute(
"""
WITH predicate AS
  (SELECT d0.trace_id,
          d0.domain,
          d0.intervals,
          d0.arg_1_id AS "?d"
   FROM DATA d0
   WHERE d0.predicate='in_motion'
     AND (d0.arg_1_type='dodgeball')),
     negated_predicate AS
  (SELECT d1.trace_id,
          d1.domain,
          d1.intervals,
          d1.arg_1_id AS "?t"
   FROM DATA d1
   WHERE d1.predicate='agent_holds'
     AND (d1.arg_1_type='teddy_bear')),
     missing_negated_predicate AS
  (SELECT trace_length_and_domains.trace_id AS trace_id,
          o0.object_id AS "?t",
          o1.object_id AS "?d"
   FROM trace_length_and_domains
   JOIN object_type_to_id o0 ON trace_length_and_domains.domain = o0.domain
   AND o0.type = 'teddy_bear'
   JOIN object_type_to_id o1 ON trace_length_and_domains.domain = o1.domain
   AND o1.type = 'dodgeball'
   WHERE NOT EXISTS
       (SELECT 1
        FROM negated_predicate
        WHERE trace_length_and_domains.trace_id = negated_predicate.trace_id
          AND o0.object_id = negated_predicate."?t"))
  (SELECT DISTINCT(missing_negated_predicate.trace_id)
   FROM missing_negated_predicate)
UNION
  (SELECT DISTINCT(trace_id)
   FROM predicate
   WHERE bit_count(predicate.intervals) > 0
     AND (EXISTS
            (SELECT 1
             FROM negated_predicate
             WHERE predicate.trace_id = negated_predicate.trace_id
               AND negated_predicate."?t" IS NOT NULL )
          OR EXISTS
            (SELECT 1
             FROM missing_negated_predicate
             WHERE predicate.trace_id = missing_negated_predicate.trace_id
               AND missing_negated_predicate."?t" IS NOT NULL )) )
UNION
  (SELECT DISTINCT(negated_predicate.trace_id)
   FROM negated_predicate
   WHERE bit_count(negated_predicate.intervals) > 0
     AND negated_predicate."?t" IS NOT NULL )


""").fetchdf()

In [ ]:
stats.con.execute(
"""
SELECT d0.trace_id,
                d0.domain,
                d0.intervals,
                d0.arg_1_id
FROM DATA d0
WHERE d0.predicate='in_motion'
  AND (d0.arg_1_type='dodgeball')
  AND d0.trace_id = 'ShyRlIFgTWW6vqNrd3K5-createGame-rerecorded'

""").fetchdf()

In [ ]:
stats.con.execute(
"""
WITH d0 AS (SELECT d0.trace_id,
          d0.domain,
          d0.intervals,
          d0.arg_1_id AS "?r"
   FROM DATA d0
   WHERE d0.predicate='in_motion'
     AND (d0.arg_1_type='dodgeball_red')
),
d1 AS (SELECT d1.trace_id,
                d1.domain,
                d1.intervals,
                d1.arg_1_id AS "?r"
         FROM DATA d1
         WHERE d1.predicate='agent_holds'
           AND (d1.arg_1_type='dodgeball_red')
), d2 AS (SELECT d2.trace_id,
                d2.domain,
                d2.intervals,
                d2.arg_1_id,
                d2.arg_2_id AS "?r"
         FROM DATA d2
         WHERE d2.predicate='touch'
           AND (d2.arg_1_type='floor')
           AND (d2.arg_2_type='dodgeball_red')
) 

SELECT d0.trace_id, d0.intervals as intervals_0, d1.intervals as intervals_1, d2.intervals as intervals_2, bit_count(d0.intervals & (~d1.intervals))
FROM d0
LEFT JOIN d1 on d0.trace_id = d1.trace_id AND d0."?r" = d1."?r"
LEFT JOIN d2 on d0.trace_id = d2.trace_id AND d0."?r" = d2."?r"
WHERE d0.trace_id = 'SFpazSkgQ7MFSwDEa3c9-gameplay-attempt-1-rerecorded'

""").fetchdf()



In [ ]:
stats.con.execute(
"""
WITH predicate AS
  (SELECT d0.trace_id,
          d0.domain,
          d0.intervals,
          d0.arg_1_id AS "agent",
          d0.arg_2_id AS "bed"
   FROM DATA d0
   WHERE d0.predicate='adjacent'
     AND (d0.arg_1_type='agent')
     AND (d0.arg_2_type='bed')),
     negated_predicate AS
  (SELECT d1.trace_id,
          d1.domain,
          d1.intervals,
          d1.arg_1_id AS "?v1",
          d1.arg_2_id AS "?v2"
   FROM DATA d1
   WHERE d1.predicate='touch'
     AND (d1.arg_1_type='dodgeball')
     AND (d1.arg_2_type='pillow')),
     missing_negated_predicate AS
  (SELECT predicate.trace_id AS trace_id,
          o0.object_id AS "?v1",
          o1.object_id AS "bed",
          o2.object_id AS "agent",
          o3.object_id AS "?v2"
   FROM predicate
   JOIN object_type_to_id o0 ON predicate.domain = o0.domain
   AND o0.type = 'dodgeball'
   JOIN object_type_to_id o1 ON predicate.domain = o1.domain
   AND o1.type = 'bed'
   AND o1.object_id = predicate."bed"
   JOIN object_type_to_id o2 ON predicate.domain = o2.domain
   AND o2.type = 'agent'
   AND o2.object_id = predicate."agent"
   JOIN object_type_to_id o3 ON predicate.domain = o3.domain
   AND o3.type = 'pillow'
   WHERE NOT EXISTS
       (SELECT 1
        FROM negated_predicate
        WHERE predicate.trace_id = negated_predicate.trace_id
          AND (o0.object_id = negated_predicate."?v1"
               OR negated_predicate."?v1" IS NULL)
          AND (o3.object_id = negated_predicate."?v2"
               OR negated_predicate."?v2" IS NULL))
     AND NOT EXISTS
       (SELECT 1
        FROM negated_predicate
        WHERE predicate.trace_id = negated_predicate.trace_id
          AND bit_count(~negated_predicate.intervals) = 0))
SELECT DISTINCT(trace_id)
FROM predicate
WHERE bit_count(predicate.intervals) > 0
  AND (EXISTS
         (SELECT *
          FROM negated_predicate
          WHERE predicate.trace_id = negated_predicate.trace_id
            AND
            AND bit_count(predicate.intervals) > bit_count(predicate.intervals & negated_predicate.intervals) )
       OR EXISTS
         (SELECT *
          FROM missing_negated_predicate
          WHERE predicate.trace_id = missing_negated_predicate.trace_id ))

""").fetchdf()

In [ ]:
stats.con.execute(
"""
WITH t283 AS
        (SELECT trace_length_and_domains.trace_id AS trace_id,
                trace_length_and_domains.domain AS DOMAIN,
                trace_length_and_domains.intervals AS intervals,
                object_assignments."?v2" AS "?v2",
                object_assignments."bed" AS "bed"
         FROM trace_length_and_domains
         JOIN
           (WITH t281 AS
              (SELECT DOMAIN,
                      object_id AS "?v2"
               FROM object_type_to_id
               WHERE TYPE = 'pillow'),
                 t282 AS
              (SELECT DOMAIN,
                      object_id AS "bed"
               FROM object_type_to_id
               WHERE TYPE = 'bed') SELECT t281.domain,
                                          t281."?v2" AS "?v2",
                                          t282."bed" AS "bed"
            FROM t281
            JOIN t282 ON (t281.domain = t282.domain)
            AND (t281."?v2" != t282."bed")) AS object_assignments ON (trace_length_and_domains.domain = object_assignments.domain)),
           t284 AS
        (SELECT trace_id,
                DOMAIN,
                intervals,
                arg_1_id AS "bed",
                arg_2_id AS "?v2"
         FROM DATA
         WHERE predicate='on'
           AND (arg_1_type='bed')
           AND (arg_2_type='pillow')) 
SELECT t283.trace_id AS trace_id,
        t283.domain AS DOMAIN,
        t283."?v2" AS "?v2",
        t283."bed" AS "bed",
        (~(t283.intervals | COALESCE(t284.intervals, t283.intervals))) AS intervals,
        bit_count(~(t283.intervals | COALESCE(t284.intervals, t283.intervals))) AS bit_count
FROM t283
LEFT JOIN t284 ON t283."trace_id"=t284."trace_id"
AND t283."?v2"=t284."?v2"
AND t283."bed"=t284."bed"
WHERE t283.trace_id = '1El1CmicSoKZKTLe8NpP-gameplay-attempt-1-rerecorded'

""").fetchdf()

In [ ]:
stats.con.execute(
"""
WITH t274 AS
     (WITH t268 AS
        (WITH t266 AS
           (SELECT trace_length_and_domains.trace_id AS trace_id,
                   trace_length_and_domains.domain AS DOMAIN,
                   trace_length_and_domains.intervals AS intervals,
                   object_assignments."?v1" AS "?v1",
                   object_assignments."?v2" AS "?v2"
            FROM trace_length_and_domains
            JOIN
              (WITH t264 AS
                 (SELECT DOMAIN,
                         object_id AS "?v1"
                  FROM object_type_to_id
                  WHERE TYPE = 'dodgeball'),
                    t265 AS
                 (SELECT DOMAIN,
                         object_id AS "?v2"
                  FROM object_type_to_id
                  WHERE TYPE = 'pillow') SELECT t264.domain,
                                                t264."?v1" AS "?v1",
                                                t265."?v2" AS "?v2"
               FROM t264
               JOIN t265 ON (t264.domain = t265.domain)
               AND (t264."?v1" != t265."?v2")) AS object_assignments ON (trace_length_and_domains.domain = object_assignments.domain)),
              t267 AS
           (SELECT trace_id,
                   DOMAIN,
                   intervals,
                   arg_1_id AS "?v1",
                   arg_2_id AS "?v2"
            FROM DATA
            WHERE predicate='touch'
              AND (arg_1_type='dodgeball')
              AND (arg_2_type='pillow')) SELECT t266.trace_id AS trace_id,
                                                t266.domain AS DOMAIN,
                                                t266."?v1" AS "?v1",
                                                t266."?v2" AS "?v2",
                                                (~(t266.intervals | COALESCE(t267.intervals, t266.intervals))) AS intervals
         FROM t266
         LEFT JOIN t267 ON t266."trace_id"=t267."trace_id"
         AND t266."?v1"=t267."?v1"
         AND t266."?v2"=t267."?v2") SELECT *
      FROM t268
      WHERE bit_count(intervals) != 0),
        t275 AS
     (WITH t273 AS
        (WITH t271 AS
           (SELECT trace_length_and_domains.trace_id AS trace_id,
                   trace_length_and_domains.domain AS DOMAIN,
                   trace_length_and_domains.intervals AS intervals,
                   object_assignments."?v2" AS "?v2",
                   object_assignments."bed" AS "bed"
            FROM trace_length_and_domains
            JOIN
              (WITH t269 AS
                 (SELECT DOMAIN,
                         object_id AS "?v2"
                  FROM object_type_to_id
                  WHERE TYPE = 'pillow'),
                    t270 AS
                 (SELECT DOMAIN,
                         object_id AS "bed"
                  FROM object_type_to_id
                  WHERE TYPE = 'bed') SELECT t269.domain,
                                             t269."?v2" AS "?v2",
                                             t270."bed" AS "bed"
               FROM t269
               JOIN t270 ON (t269.domain = t270.domain)
               AND (t269."?v2" != t270."bed")) AS object_assignments ON (trace_length_and_domains.domain = object_assignments.domain)),
              t272 AS
           (SELECT trace_id,
                   DOMAIN,
                   intervals,
                   arg_1_id AS "bed",
                   arg_2_id AS "?v2"
            FROM DATA
            WHERE predicate='on'
              AND (arg_1_type='bed')
              AND (arg_2_type='pillow')) SELECT t271.trace_id AS trace_id,
                                                t271.domain AS DOMAIN,
                                                t271."?v2" AS "?v2",
                                                t271."bed" AS "bed",
                                                (~(t271.intervals | COALESCE(t272.intervals, t271.intervals))) AS intervals
         FROM t271
         LEFT JOIN t272 ON t271."trace_id"=t272."trace_id"
         AND t271."?v2"=t272."?v2"
         AND t271."bed"=t272."bed") SELECT *
      FROM t273) 
      

SELECT t274.trace_id,
      t274.domain,
      t274."?v1",
      t274."?v2",
      t275."bed",
      (t274.intervals | t275.intervals) AS intervals,
      bit_count(t274.intervals | t275.intervals) AS bit_count
FROM t274
LEFT JOIN t275 ON (t274.trace_id=t275.trace_id)
WHERE t274.trace_id = '1aTng0m9240WEAU975WE-gameplay-attempt-2-rerecorded'

""").fetchdf()




In [ ]:
# duckdb.sql("""SELECT * FROM data WHERE predicate='touch' AND arg_1_is_game_object IS TRUE AND arg_2_is_game_object IS TRUE""").df()
new_trace_ids = stats.con.execute(
"""
WITH positive_predicate AS (
    SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "bed",
            arg_2_id AS "?v1"
    FROM
            data
    WHERE
            predicate = 'on'
            AND (arg_1_type = 'bed')
            AND (arg_2_type = 'dodgeball')
),
negative_predicate AS (
    SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "?v1",
            arg_2_id AS "?v2"
    FROM
            data
    WHERE
            predicate = 'touch'
            AND (arg_1_type = 'dodgeball')
            AND (arg_2_type = 'pillow')

),
missing_negative_predicates AS (
    SELECT 
            positive_predicate.trace_id as trace_id,
    FROM
            positive_predicate
    JOIN 
            object_type_to_id o1
    ON
            positive_predicate.domain = o1.domain
            AND o1.type='dodgeball' 
            AND o1.object_id = positive_predicate."?v1"
    JOIN 
            object_type_to_id o2
    ON
            positive_predicate.domain = o2.domain
            AND o2.type='pillow'
    JOIN 
            negative_predicate    
    ON 
            positive_predicate.trace_id = negative_predicate.trace_id
            AND o1.object_id != negative_predicate."?v1"
            AND o2.object_id != negative_predicate."?v2"
)
SELECT DISTINCT(trace_id) FROM positive_predicate
WHERE NOT EXISTS (
    SELECT * FROM negative_predicate
    WHERE
            positive_predicate.trace_id = negative_predicate.trace_id
            AND positive_predicate."?v1" = negative_predicate."?v1"
            AND bit_count(positive_predicate.intervals) = bit_count(positive_predicate.intervals & negative_predicate.intervals)
) OR EXISTS (
    SELECT * FROM missing_negative_predicates
    WHERE
            positive_predicate.trace_id = missing_negative_predicates.trace_id
)


;
""").fetchall()

new_trace_ids = set(itertools.chain.from_iterable(new_trace_ids))
print(len(new_trace_ids))
current_results.symmetric_difference(new_trace_ids)

In [ ]:
refactored_query = """
SELECT count(*) from trace_length_and_domains
""".strip()

stats.con.execute(refactored_query).df()


In [ ]:
refactored_query = """
SELECT * FROM (
SELECT
            trace_id, 
            count(trace_id) as trace_count,
            domain
            /*
            domain,
            intervals,
            arg_1_id AS "bed",
            arg_2_id AS "?v1"
            */
    FROM
            data
    WHERE
            predicate = 'touch'
            AND (arg_1_type = 'pillow')
            AND (arg_2_type = 'dodgeball')
        GROUP BY trace_id, domain
) WHERE 
(domain = 'few' AND trace_count = 2) OR 
(domain = 'medium' AND trace_count = 1) OR
(domain = 'many' AND trace_count = 3)
""".strip()

for t in stats.con.execute(refactored_query).df().trace_id:
    print(t)



In [ ]:
q_neg = """
SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "?v1",
    FROM
            data
    WHERE
            predicate = 'agent_crouches'
            
"""

stats.con.execute(q_neg).df()

In [ ]:
q_missing_neg = """
WITH positive_predicate AS (
    SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "bed",
            arg_2_id AS "?v1"
    FROM
            data
    WHERE
            predicate = 'on'
            AND (arg_1_type = 'bed')
            AND (arg_2_type = 'dodgeball')
            AND (trace_id = '4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded')
),
negative_predicate AS (
    SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "?v1",
    FROM
            data
    WHERE
            predicate = 'touch'
            AND (arg_1_type = 'dodgeball')
            AND (arg_2_type = 'pillow')
            AND (trace_id = '4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded')

),
missing_negative_predicates AS (
    SELECT 
            positive_predicate.trace_id as trace_id,
            object_type_to_id.object_id as "?v1"
    FROM
            positive_predicate
    JOIN 
            object_type_to_id
    ON
            positive_predicate.domain = object_type_to_id.domain
            AND object_type_to_id.type='pillow'
    JOIN 
            negative_predicate    
    ON 
            positive_predicate.trace_id = negative_predicate.trace_id
            AND object_type_to_id.object_id != negative_predicate."?v1"
)
SELECT * FROM missing_negative_predicates 
WHERE (trace_id = '4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded')
"""

stats.con.execute(q_missing_neg).df()

In [ ]:
q_joint = """
WITH positive_predicate AS (
    SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "bed",
            arg_2_id AS "?v1"
    FROM
            data
    WHERE
            predicate = 'on'
            AND (arg_1_type = 'bed')
            AND (arg_2_type = 'dodgeball')
            AND (trace_id = '4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded')
),
negative_predicate AS (
    SELECT
            trace_id,
            domain,
            intervals,
            arg_1_id AS "?v1",
    FROM
            data
    WHERE
            predicate = 'touch'
            AND (arg_1_type = 'dodgeball')
            AND (arg_2_type = 'pillow')
            AND (trace_id = '4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded')
)
SELECT bit_count(positive_predicate.intervals & (~ negative_predicate.intervals)) 
FROM positive_predicate
JOIN negative_predicate
ON positive_predicate.trace_id = negative_predicate.trace_id
AND positive_predicate."?v1" = negative_predicate."?v1"
"""

stats.con.execute(q_joint).df()

In [ ]:
old_trace_ids = stats.con.execute("""WITH t10 AS (
        WITH t8 AS (
                SELECT
                        trace_id,
                        domain,
                        intervals,
                        arg_1_id AS "bed",
                        arg_2_id AS "?v1"
                FROM
                        data
                WHERE
                        predicate = 'on'
                        AND (arg_1_type = 'bed')
                        AND (arg_2_type = 'dodgeball')
        ),
        t9 AS (
                WITH t7 AS (
                        WITH t5 AS (
                                SELECT
                                        trace_length_and_domains.trace_id AS trace_id,
                                        trace_length_and_domains.domain AS domain,
                                        trace_length_and_domains.intervals AS intervals,
                                        object_assignments."?v2" as "?v2",
                                        object_assignments."?v1" as "?v1"
                                FROM
                                        trace_length_and_domains
                                        JOIN (
                                                WITH t3 AS (
                                                        SELECT
                                                                domain,
                                                                object_id AS "?v2"
                                                        FROM
                                                                object_type_to_id
                                                        WHERE
                                                                type = 'pillow'
                                                ),
                                                t4 AS (
                                                        SELECT
                                                                domain,
                                                                object_id AS "?v1"
                                                        FROM
                                                                object_type_to_id
                                                        WHERE
                                                                type = 'dodgeball'
                                                )
                                                SELECT
                                                        t3.domain,
                                                        t3."?v2" AS "?v2",
                                                        t4."?v1" AS "?v1"
                                                FROM
                                                        t3
                                                        JOIN t4 ON (t3.domain = t4.domain)
                                                        AND (t3."?v2" != t4."?v1")
                                        ) AS object_assignments ON (
                                                trace_length_and_domains.domain = object_assignments.domain
                                        )
                        ),
                        t6 AS (
                                SELECT
                                        trace_id,
                                        domain,
                                        intervals,
                                        arg_1_id AS "?v1",
                                        arg_2_id AS "?v2"
                                FROM
                                        data
                                WHERE
                                        predicate = 'touch'
                                        AND (arg_1_type = 'dodgeball')
                                        AND (arg_2_type = 'pillow')
                        )
                        SELECT
                                t5.trace_id as trace_id,
                                t5.domain as domain,
                                t5."?v2" as "?v2",
                                t5."?v1" as "?v1",
                                (
                                        ~(
                                                t5.intervals | COALESCE(t6.intervals, t5.intervals)
                                        )
                                ) AS intervals
                        FROM
                                t5
                                LEFT JOIN t6 ON t5."trace_id" = t6."trace_id"
                                AND t5."?v2" = t6."?v2"
                                AND t5."?v1" = t6."?v1"
                )
                SELECT
                        *
                FROM
                        t7
                WHERE
                        bit_count(intervals) != 0
        )
        SELECT
                t8.trace_id,
                t8.domain,
                t8."bed",
                t8."?v1",
                t9."?v2",
                (t8.intervals & t9.intervals) AS intervals
        FROM
                t8
                INNER JOIN t9 ON (t8.trace_id = t9.trace_id)
                AND (t9."?v1" = t8."?v1")
)
SELECT
        DISTINCT trace_id
FROM
        t10
WHERE
        bit_count(intervals) != 0;""").fetchall()

old_trace_ids = set(itertools.chain.from_iterable(old_trace_ids))
old_trace_ids

In [ ]:
new_trace_ids - old_trace_ids

In [ ]:
stats.con.execute(
"""
SELECT * FROM DATA
WHERE trace_id='4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded' AND predicate='on' AND (arg_1_type = 'bed') AND (arg_2_type = 'dodgeball')
                  
""").fetchdf()

In [ ]:
stats.con.execute(
"""
SELECT * FROM DATA
WHERE trace_id='4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded' AND predicate='touch' AND (arg_1_type = 'dodgeball') AND (arg_2_type = 'pillow')
                  
""").fetchdf()

In [ ]:
stats.con.execute(
"""
WITH ball_touch_pillow AS (
SELECT * FROM DATA
WHERE trace_id='4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded' AND predicate='touch' AND (arg_1_type = 'dodgeball') AND (arg_2_type = 'pillow')
),
ball_on_bed AS (
SELECT * FROM DATA
WHERE trace_id='4WUtnD8W6PGVy0WBtVm4-editGame-rerecorded' AND predicate='on' AND (arg_1_type = 'bed') AND (arg_2_type = 'dodgeball')
)
SELECT bit_count(ball_on_bed.intervals & (~ball_touch_pillow.intervals)) FROM ball_on_bed
JOIN ball_touch_pillow
ON ball_on_bed.arg_2_id = ball_touch_pillow.arg_1_id
                  
""").fetchdf()


In [ ]:
stats.con.execute(
"""
SELECT * FROM object_type_to_id
WHERE type='pillow'             
""").fetchdf()

In [ ]:
duckdb.sql("SELECT SUM(LENGTH(intervals)) FROM data WHERE predicate not in ('equal_x_position', 'equal_z_position');")

In [ ]:
con = duckdb.connect(database=':memory:')


In [ ]:
con.execute("SHOW TABLES;")

In [ ]:
refactored_query = """
SELECT trace_id, arg_1_id, arg_2_id, overlap, d1_count, d2_count
FROM (WITH 
d1 AS (SELECT trace_id, arg_1_id, arg_1_type, arg_2_id, arg_2_type, intervals FROM data WHERE predicate='on' AND arg_1_type!='building' AND arg_2_type!='building'), 
d2 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='on' AND arg_1_type!='building' AND arg_2_type!='building')
SELECT d1.trace_id, d1.arg_1_id, d1.arg_1_type, d1.arg_2_id, d1.arg_2_type, bit_count(d1.intervals & d2.intervals) as overlap, bit_count(d1.intervals) as d1_count, bit_count(d2.intervals) as d2_count
FROM d1
INNER JOIN d2 ON d1.trace_id = d2.trace_id AND d1.arg_1_id = d2.arg_2_id AND d1.arg_2_id = d2.arg_1_id)
WHERE overlap > 0
"""
reciprocal_on_df = duckdb.sql(refactored_query).df()  # .to_csv('temp_outputs/a_on_b_and_b_on_a.csv')
reciprocal_on_df
# reciprocal_on_df = reciprocal_on_df.assign(remove_d1=reciprocal_on_df.d1_count <= reciprocal_on_df.d2_count)
# reciprocal_on_df


In [ ]:
duckdb.sql('CREATE INDEX idx_data_arg_1_type ON data (arg_1_type)')

In [ ]:
predicate_on_rows = bitstrings_df.predicate == 'on'
indices_to_remove = []

for _, row in reciprocal_on_df.iterrows():
    row_filter = predicate_on_rows & (bitstrings_df.trace_id == row.trace_id)
    if row.remove_d1:
        row_filter &= (bitstrings_df.arg_1_id == row.arg_1_id) & (bitstrings_df.arg_2_id == row.arg_2_id)

    else:
        row_filter &= (bitstrings_df.arg_1_id == row.arg_2_id) & (bitstrings_df.arg_2_id == row.arg_1_id)

    indices_to_remove.extend(bitstrings_df[row_filter].index)


print(indices_to_remove)



In [ ]:
len(set(indices_to_remove))

In [ ]:
# filtered_bitstrings_df = bitstrings_df.drop(index=indices_to_remove)
# filtered_bitstrings_df = filtered_bitstrings_df.reset_index(drop=True)

# print(filtered_bitstrings_df.shape, bitstrings_df.shape, filtered_bitstrings_df.index.max())

# filtered_bitstrings_df.to_pickle(bitstings_df_path)

In [ ]:
refactored_query = """
SELECT trace_id, arg_1_id, arg_2_id, overlap  
FROM (WITH 
d1 AS (SELECT trace_id, arg_1_id, arg_1_type, intervals FROM data WHERE predicate='agent_holds'), 
d2 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='adjacent' AND (arg_1_type='agent' OR arg_2_type='agent'))
SELECT d2.trace_id, d2.arg_1_id, d2.arg_2_id, bit_count(d1.intervals & d2.intervals) as overlap
FROM d2
INNER JOIN d1 ON d1.trace_id = d2.trace_id AND (d1.arg_1_id = d2.arg_1_id OR d1.arg_1_id = d2.arg_2_id)
)
WHERE overlap > 0
"""
duckdb.sql(refactored_query).df()  # .to_csv('temp_outputs/a_on_b_and_b_on_a.csv')


In [ ]:
refactored_query = """
SELECT count(*) FROM data
WHERE predicate='agent_holds'
"""
duckdb.sql(refactored_query)  # .df()  # .to_csv('temp_outputs/a_on_b_and_b_on_a.csv')

In [ ]:
in_arg_types = duckdb.sql("SELECT arg_types.* FROM (SELECT DISTINCT(arg_1_type, arg_2_type) as arg_types FROM data WHERE predicate='in');").fetchall()
in_arg_types = [tuple(x) for x in in_arg_types]
first_arg_types, second_arg_types = zip(*in_arg_types)

In [ ]:
[t for t in in_arg_types if t[0] == 'mug']

In [ ]:
on_arg_types = duckdb.sql("SELECT arg_types.* FROM (SELECT DISTINCT(arg_1_type, arg_2_type) as arg_types FROM data WHERE predicate='on');").fetchall()
on_arg_types = [tuple(x) for x in on_arg_types]
on_first_arg_types, on_second_arg_types = zip(*on_arg_types)

In [ ]:
[t for t in on_arg_types if t[1] == 'desk']

In [ ]:
import room_and_object_types

on_types_by_category = defaultdict(set)
for t in on_first_arg_types:
    on_types_by_category[room_and_object_types.TYPES_TO_CATEGORIES[t]].add(t)


for cat in on_types_by_category:
    print(cat)
    print(on_types_by_category[cat])
    print()

In [ ]:
refactored_query = """
SELECT * FROM (WITH 
d1 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='on' AND arg_1_type!='building' AND arg_2_type!='building'), 
d2 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='on' AND arg_1_type!='building' AND arg_2_type!='building')
SELECT d1.trace_id, d1.arg_1_id, d1.arg_2_id, bit_count(d1.intervals & d2.intervals) as overlap
FROM d1
INNER JOIN d2 ON d1.trace_id = d2.trace_id AND d1.arg_1_id = d2.arg_2_id AND d1.arg_2_id = d2.arg_1_id)
WHERE overlap > 100
"""
duckdb.sql(refactored_query).df().to_csv('temp_outputs/a_on_b_and_b_on_a.csv')

# duckdb.sql("SELECT * FROM data")

In [ ]:
refactored_query = """
SELECT * FROM (WITH 
d1 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='on' AND arg_1_type!='building' AND arg_2_type!='building'), 
d2 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='in' AND arg_1_type!='building' AND arg_2_type!='building')
SELECT d1.trace_id, d1.arg_1_id, d1.arg_2_id, bit_count(d1.intervals & d2.intervals) as overlap
FROM d1
INNER JOIN d2 ON d1.trace_id = d2.trace_id AND d1.arg_1_id = d2.arg_2_id AND d1.arg_2_id = d2.arg_1_id)
WHERE overlap > 100
"""
duckdb.sql(refactored_query).df().to_csv('temp_outputs/a_on_b_and_b_in_a.csv')

# duckdb.sql("SELECT * FROM data")

In [ ]:
refactored_query = """
SELECT trace_id, domain, arg_1_id, arg_2_id, bit_position('1'::BIT, joint_intervals) as "first_index" FROM (WITH 
d1 AS (SELECT trace_id, domain, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='on' AND (arg_1_type='building' OR arg_2_type='building')), 
d2 AS (SELECT trace_id, arg_1_id, arg_2_id, intervals FROM data WHERE predicate='on' AND (arg_1_type='building' OR arg_2_type='building'))
SELECT d1.trace_id, d1.domain, d1.arg_1_id, d1.arg_2_id, d1.intervals & d2.intervals as joint_intervals
FROM d1
INNER JOIN d2 ON d1.trace_id = d2.trace_id AND d1.arg_1_id = d2.arg_2_id AND d1.arg_2_id = d2.arg_1_id)
WHERE bit_count(joint_intervals) > 100
"""
buildings_df = duckdb.sql(refactored_query).df().to_csv('temp_outputs/a_on_b_and_b_on_a_buildings.csv')
buildings_df
# buildings_df[(buildings_df.trace_id == 'Q6a8AbiIdcLA9tJzAu14-createGame-rerecorded') & (buildings_df.arg_2_id == 'SmallSlide|-00.81|+00.14|-03.10')]

# duckdb.sql("SELECT * FROM data")

In [ ]:
refactored_query = """
SELECT * FROM data
WHERE trace_id='Q6a8AbiIdcLA9tJzAu14-createGame-rerecorded' AND arg_1_id='building_1' and arg_2_id='SmallSlide|-00.81|+00.14|-03.10' AND predicate='on'
"""

d = duckdb.sql(refactored_query).df()
print(d.loc[0, 'intervals'] == d.loc[1, 'intervals'])
d

In [ ]:
refactored_query = """
SELECT trace_id, domain, arg_1_id, arg_2_id, bit_position('1'::BIT, intervals) as 'first_index' FROM data
WHERE predicate='on' and arg_2_type in ('bed', 'desk') and arg_1_type NOT IN ('floor', 'rug')
"""
duckdb.sql(refactored_query).df().to_csv('temp_outputs/bed_or_desk_on_object_that_is_not_floor_or_rug.csv')

# duckdb.sql("SELECT * FROM data")

In [ ]:
p = game_asts[1][4][1].preferences[0].definition.forall_pref.preferences.pref_body.body.then_funcs[0].seq_func.once_pred
print(p.keys())
ast_printer.ast_section_to_string(p, ast_parser.PREFERENCES)

In [ ]:
refactored_query = stats.filter(p, {"?b": ["ball"], "?t": ["hexagonal_bin"]})
print(refactored_query)

In [ ]:
duckdb.sql('PRAGMA force_index_join;')
duckdb.sql("PRAGMA explain_output='OPTIMIZED_ONLY';")

In [ ]:
# q = """
#     SELECT t0.trace_id, t0.domain, t0."?b", t1."door", t1."agent", (t0.intervals & t1.intervals) AS intervals
#     FROM (SELECT trace_id, domain, intervals, arg_1_id AS "?b" FROM data WHERE predicate='agent_holds' AND (arg_1_type IN ('beachball'::arg_type, 'basketball'::arg_type, 'dodgeball'::arg_type, 'golfball'::arg_type))) as t0
#     INNER JOIN (SELECT trace_id, domain, intervals, arg_1_id AS "door", arg_2_id AS "agent" FROM data WHERE predicate='adjacent' AND (arg_1_type='door') AND (arg_2_type='agent')) as t1
#     ON (t0.trace_id=t1.trace_id)
# """

refactored_query = """
    SELECT t0.trace_id, t0.domain, t0.arg_1_id AS "?b", t1.arg_1_id AS "door", t1.arg_2_id AS "agent", (t0.intervals & t1.intervals) AS intervals
    FROM data AS t0
    INNER JOIN data AS t1
    ON (t0.trace_id=t1.trace_id)
    WHERE t0.predicate='agent_holds' AND (t0.arg_1_type IN ('beachball'::arg_type, 'basketball'::arg_type, 'dodgeball'::arg_type, 'golfball'::arg_type)) AND
    t1.predicate='adjacent' AND (t1.arg_1_type='door') AND (t1.arg_2_type='agent')

"""

print(duckdb.sql(f"EXPLAIN ANALYZE ({refactored_query})").fetchone()[1])

In [ ]:
duckdb.sql("INSERT INTO domains VALUES ('few'), ('medium'), ('many')")

In [ ]:
df = duckdb.sql(refactored_query).fetchdf()

In [ ]:
duckdb.sql('CREATE INDEX idx_data_predicate ON data (predicate)')

In [ ]:
t = np.zeros(10, dtype=np.uint8)
t[1:4] = 1
b = t.tobytes()
d = {b[0]: '0', b[1]: '1'}

In [ ]:
df[df.trace_id == '1El1CmicSoKZKTLe8NpP-preCreateGame-rerecorded']

In [ ]:
bits_from_df = np.unpackbits(np.frombuffer(interval_from_df, dtype=np.uint8))
len(bits_from_df), bits_from_df

In [ ]:
results = duckdb.sql(refactored_query).fetchall()

for tup in results:
    sub_df = df[(tup[0] == df['trace_id']) & (tup[2] == df['?b']) & (tup[3] == df['?t'])]
    if len(sub_df) != 1:
        print(f'Error: {tup[:-1]}')
        print(len(sub_df))
        break

    expected_length = duckdb.sql(f"SELECT length from trace_length_and_domains WHERE trace_id='{tup[0]}'").fetchone()[0]

    bits_from_df = np.unpackbits(np.frombuffer(sub_df.intervals.item(), dtype=np.uint8))
    bits_from_db = np.fromiter(map(int, tup[-1]), dtype=np.uint8)

    if len(bits_from_db) != expected_length:
        print(f'Error: {tup[:-1]}')
        print(len(bits_from_db))
        print(expected_length)
        print(len(bits_from_df))
        break

    if not np.all(bits_from_df[-expected_length:] == bits_from_db):
        print(f'Error: {tup[:-1]}')
        print(np.where(bits_from_df[-expected_length:] != bits_from_db))


In [ ]:
tup[:-1]

In [ ]:
len(bits_from_df[-550:])

In [ ]:
bits_from_db = np.fromiter(map(int, t[-1]), dtype=np.uint8)

In [ ]:
bits_from_df = np.unpackbits(np.frombuffer(df[df.trace_id == t[0]].intervals.item(), dtype=np.uint8))

In [ ]:
print(len(bits_from_db), len(bits_from_df))

In [ ]:
1264 % 16

In [ ]:
print(len(bits_from_db), len(bits_from_df))
np.where(bits_from_db[:] != bits_from_df[14:])

In [ ]:
bits_from_db[500:540], bits_from_df[500:540]

In [ ]:
(bits_from_db[:min_length] != bits_from_df[:min_length])[:10]

## Gameplan from here

* For (setup, each preference):
* Inspect to make sure all predicates implemeneted in the cache, it doesn't use a once-measure (and probably also not a hold-while, at least at first), etc. 
    * otherwise default to the basic implementation
* If we're go to run on a particular thing:
    * If it's a setup, it's trivial
    * If it's a preference, check if it's an at-end or then
        * If it's an at-end, it's trivial (can probably fold this into the query by doing `get_bit(intervals, length - 1)` if we join on the trace lengths table
        * If it's a then, enumerate over the predicates of each modal, and query for them
            * For each one, fetch the df for the query, use the trace lengths to transform the intervals to the expected format
            * Add the index of the modal to the df
            * Enumerate through all trace ids and assignments, and for each assignment where we have all modals represented: 
                * Create the joint state interval
                * Iterate through the joint using the state machine logic
                * Count satisfactions
                * ...
                * Profit!

In [ ]:
import tatsu.ast
import tatsu.grammars
from ast_parser import ASTParser, SECTION_CONTEXT_KEY, VARIABLES_CONTEXT_KEY
from ast_utils import simplified_context_deepcopy, deepcopy_ast, ASTCopyType, replace_child


DEFAULT_UNSUPPORTED_RULES = [
    'function_comparison',
    'function_eval',
    'predicate_adjacent_side_3',
    'predicate_adjacent_side_4',
    'predicate_between',
    'predicate_faces',
    'predicate_is_setup_object',
    'predicate_opposite',
    'predicate_rug_color_under',
    'predicate_same_color',
    'predicate_same_object',
    'predicate_same_type',
    'super_predicate_exists',
    'super_predicate_forall',
    'once_measure',
    'while_hold',
]


def _pref_forall_pos_to_key(pos: int):
    return f'pref_forall_{pos}'


class MixedTraceFilterGameParser(ASTParser):
    unsupported_rules: typing.Set[str]

    def __init__(self, unsupported_rules: typing.Sequence[str] = DEFAULT_UNSUPPORTED_RULES):
        super().__init__()
        self.expected_keys = set()
        self.unsupported_rules = set(unsupported_rules)

    def __call__(self, ast, **kwargs):
        initial_call = 'inner_call' not in kwargs or not kwargs['inner_call']
        if initial_call:
            kwargs['inner_call'] = True
            kwargs['local_context'] = {'mapping': {VARIABLES_CONTEXT_KEY: {}}}
            kwargs['global_context'] = {}
            self.expected_keys = set()
            self.unsupported_keys = set()
            # self.traces_by_preference_or_section = {}
            # self.preferences_or_sections_with_implemented_predicates = set()
            # self.predicate_strings_by_preference_or_section = defaultdict(set)
            # self.not_implemented_predicate_counts = defaultdict(int)

        retval = super().__call__(ast, **kwargs)

        if initial_call:
            return self.unsupported_keys, self.expected_keys
        else:
            return retval

    def _current_ast_to_contexts_hook(self, ast: tatsu.ast.AST, kwargs: typing.Dict[str, typing.Any]):
        rule = typing.cast(str, ast.parseinfo.rule)  # type: ignore

        if rule == 'pref_forall':
            kwargs['local_context']['current_pref_forall_index'] = ast.parseinfo.pos

        if rule == 'preference':
            kwargs['local_context']['current_preference_name'] = ast.pref_name

    def _handle_ast(self, ast: tatsu.ast.AST, **kwargs):
        self._current_ast_to_contexts(ast, **kwargs)
        kwargs['local_context']['mapping'] = ast_parser.update_context_variables(ast, kwargs['local_context']['mapping'])

        current_key = None
        if SECTION_CONTEXT_KEY in kwargs and kwargs[SECTION_CONTEXT_KEY] == ast_parser.SETUP:
            current_key = kwargs[SECTION_CONTEXT_KEY]
        elif 'current_pref_forall_index' in kwargs['local_context']:
            current_key =_pref_forall_pos_to_key(kwargs['local_context']['current_pref_forall_index'])
        elif 'current_preference_name' in kwargs['local_context']:
            current_key = kwargs['local_context']['current_preference_name']
        
        if current_key is not None:
            self.expected_keys.add(current_key)

            if ast.parseinfo.rule in self.unsupported_rules:
                self.unsupported_keys.add(current_key)

        for key in ast:
            if key != 'parseinfo':
                child_kwargs = simplified_context_deepcopy(kwargs)
                retval = self(ast[key], **child_kwargs)
                self._update_contexts_from_retval(kwargs, retval)
            
            


In [ ]:
game_parser = MixedTraceFilterGameParser()
for ast in game_asts:
    unsupported, expected = game_parser(ast)
    supported = expected - unsupported
    print(f'Game {ast[1].game_name} has supported keys: {list(supported)} and unsupported keys: {list(unsupported)}')

In [ ]:



DUMMY_PREFERENCE_GAME = """(define (game dummy-preference-game) (:domain many-objects-room-v1)
(:constraints (and
    (preference dummyPreference
            (at-end (game-over))
    )
))
(:scoring (count dummyPreference)
))
"""


class ASTTraceFilterSplitter(ast_parser.ASTParser):
    keep_keys: typing.Set[str]
    remove_keys: typing.Set[str]
    should_insert_dummy_preference: bool
    
    def __init__(self, grammar_parser: tatsu.grammars.Grammar):
        self.grammar_parser = grammar_parser

    def __call__(self, ast, **kwargs):
        initial_call = 'inner_call' not in kwargs or not kwargs['inner_call']
        if initial_call:
            kwargs['inner_call'] = True
            
            if 'remove_keys' not in kwargs:
                raise ValueError('remove_keys must be specified')
            self.remove_keys = kwargs['remove_keys']

            if len(self.remove_keys) == 0:
                raise ValueError('remove_keys must be non-empty')

            if 'keep_keys' not in kwargs:
                raise ValueError('keep_keys must be specified')
            self.keep_keys = kwargs['keep_keys']

            if len(self.keep_keys) == 0:
                raise ValueError('keep_keys must be non-empty')

            ast = deepcopy_ast(ast)

            # Handle the setup right here and now, if we're removing it
            if ast_parser.SETUP in self.remove_keys:
                ast = (*ast[:3], *ast[4:])
                # If the only thin we're removing is the setup, we're done
                if len(self.remove_keys) == 1:
                    return ast

            # check if we're only keeping the setup and inserting a dummy preference, because if so, we're done
            if len(self.keep_keys) == 1 and ast_parser.SETUP in self.keep_keys:
                dummy_preference_game = self.grammar_parser.parse(DUMMY_PREFERENCE_GAME)
                return (*ast[:4], dummy_preference_game[3], *ast[4:])

        super().__call__(ast, **kwargs)

        if initial_call:
            return ast
        
    def _handle_ast(self, ast: tatsu.ast.AST, **kwargs):
        rule = ast.parseinfo.rule

        if rule == 'preferences':
            if isinstance(ast.preferences, tatsu.ast.AST):
                raise ValueError(f'If removing a single preference, the initial call should handle it, so this should never occur')
            
            new_children = typing.cast(typing.List[tatsu.ast.AST], deepcopy_ast(ast.preferences, ASTCopyType.NODE))
            indices_to_remove = []
            for i, child in enumerate(new_children):
                if child.parseinfo.rule == 'preference' and child.pref_name in self.remove_keys:
                    print(f'Removing preference {child.pref_name}')
                    indices_to_remove.append(i)
                elif child.parseinfo.rule == 'pref_forall' and _pref_forall_pos_to_key(child.parseinfo.pos) in self.remove_keys:
                    print(f'Removing pref_forall {_pref_forall_pos_to_key(child.parseinfo.pos)}')
                    indices_to_remove.append(i)

            for i in reversed(indices_to_remove):
                new_children.remove(new_children[i])

            replace_child(ast, 'preferences', new_children)

        else:
            for key in ast:
                if key != 'parseinfo':
                    self(ast[key], **kwargs)

            
        


In [ ]:
game_parser = MixedTraceFilterGameParser()
game_splitter = ASTTraceFilterSplitter(grammar_parser)  # type: ignore
ast = game_asts[0]
unsupported, expected = game_parser(ast)
supported = expected - unsupported

if len(supported) > 0 and len(unsupported) > 0:
    print(f'Game {ast[1].game_name} has supported keys: {list(supported)} and unsupported keys: {list(unsupported)}')
    supported_only = game_splitter(ast, keep_keys=supported, remove_keys=unsupported)
    unsupported_only = game_splitter(ast, keep_keys=unsupported, remove_keys=supported)

    print('=' * 80)
    print(ast_printer.ast_to_string(supported_only, '\n'))
    print('=' * 80)
    print(ast_printer.ast_to_string(unsupported_only, '\n'))

In [ ]:
from itertools import chain
categorical_type = pd.api.types.CategoricalDtype(sorted(chain.from_iterable(duckdb.sql("SELECT enum_range(NULL::trace_id)").fetchone())), ordered=True)

In [ ]:
trace_id_to_length_df = duckdb.sql('SELECT * FROM trace_length_and_domains').fetchdf()
trace_id_to_length_df.drop(columns=['domain'], inplace=True)
trace_id_to_length_df.rename(columns=dict(length='trace_length'), inplace=True)
# trace_id_to_length_df.astype(dict(trace_id=categorical_type), copy=False)
trace_id_to_length_df.head()

In [ ]:
def _df_intervals_to_array(row):
    return np.unpackbits(np.frombuffer(row['intervals'], dtype=np.uint8))[-row['trace_length']:]


In [ ]:
df.astype(dict(trace_id=categorical_type), copy=False)

In [ ]:
# df.join(trace_id_to_length_df, on=['trace_id'], how='outer', rsuffix='_r')
merged_df = df.merge(trace_id_to_length_df, on=['trace_id'], how='left')
merged_df

In [ ]:
assigned_df = df.assign(intervals=merged_df.apply(_df_intervals_to_array, axis=1))
assigned_df

In [ ]:
# assigned_df.groupby('trace_id', as_index=True, observed=False).intervals.transform(lambda l: list(np.logical_or.reduce(*l)))

series_1 = assigned_df.groupby('trace_id', as_index=True, observed=True).intervals.agg(lambda x: reduce(np.logical_or, x.values).astype(bool))

In [ ]:
series_2 = series_1.iloc[:50]

In [ ]:
merged = pd.merge(series_1, series_2, left_index=True, right_index=True, how='inner')
s = merged.agg(lambda row: np.logical_and(row['intervals_x'], row['intervals_y']), axis=1)